# Entrainement contrastif non supervisé

Aujourd'hui, une grande partie de la recherche en deep learning est tournée du côté de l'entrainement non supervisé. Pour rappel (cf [cours 4 sur les autoencodeurs](../04_Autoencodeurs/README.md)), l'entraînement non supervisé consiste à entraîner un modèle sans données labelisées. L'avantage principal de ce type d'entrainement est que ça limite énormement le coût et la main d'oeuvre nécessaires à la préparation des données. C'est ce type d'entraînement qui a propulsé les NLP sur le devant de la scène et qui permet une génération d'images impressionnantes avec Dall-E ou de vidéos avec SORA.

Nous avons vu dans le cours 5 sur les NLP la façon de faire de l'entraînement non supervisé sur du texte (il suffit de récuperer n'importe quel texte et d'entrainer le modèle à prédire le prochain caractère). Pour ce qui est du traitement d'images, nous n'avons pas vraiment vu comment faire si ce n'est une légère référence à CLIP dans le cours 7 sur les *transformers*. 

## Comment adapter l'entraînement contrastif au non supervisé ?

### Modèle CLIP

Nous avons déjà vu le modèle [CLIP](https://openai.com/index/clip/) dans [le cours 7](../07_Transformers/05_UtilisationsPossibles.ipynb) qui est entrainé de manière contrastive au sein du *batch*. C'est une approche un peu différente de ce que l'on a vu dans le notebook précédent, un *vision transformer* prend en entrée les images du *batch* et un *text transformer* prend en entrée les descriptions de chaque image. Le modèle est alors entraîné à associer correctement les images et les descriptions de manière contrastive (minimisation de la distance entre les *embeddings* de la même paire et maximisation pour les *embeddings* de paires différentes). 

![clip](./images/clip.png)

CLIP est un modèle utilisant le *loss* contrastif mais il n'est pas réellement non supervisé. Il se base sur des paires d'images/textes qui constituent les labels d'entraînement. 

### Entraînement non supervisé pour les images

Quand on parle de non supervisé, nous voulons qu'aucune donnée étiquetée ne soit requise. La méthode [SimCLR](https://arxiv.org/pdf/2002.05709) introduit une technique d'utilisation de l'apprentissage contrastif dans un cas non supervisé.   
L'idée est à nouveau de traîter un batch de données où chaque élément du batch est une paire d'image. Mais la particularité de cette paire d'image c'est qu'il s'agit de la même image sur laquelle on a appliqué une transformation (voir [cours bonus sur *data augmentation*](../Bonus_CoursSpécifiques/03_DataAugmentation.ipynb)). Chaque image est passée dans un réseau identique (siamois) et le modèle va alors être entrainé à minimiser la distance entre les representations des images provenant de la même paire et de maximiser la distance entre les images provenant des paires différentes. 

Voici l'architecture du modèle : 

![simCLR](./images/simCLR.png)

Voici des exemples de transformations appliquées aux images :  

![transform](./images/transform.png)

Les deux figures ci-dessus sont extraites de l'article original

Dans cette méthode, l'aspect *data augmentation* est crucial et il est important de ne pas négliger le panel de transformations possibles. 

Pour faire l'analogie avec les positive pairs et negative pairs : 
- **Positive pairs** : Les deux images transformées $x_i$ et $x_j$ provenant de la même image $x$.
- **Negative pairs** : Deux images transformées $x_i$ et $x'_j$ provenant d'images différentes $x$ et $x'$.

Grâce à cette méthode, le modèle est capable d'apprendre des representations pertinentes des images sans avoir besoin de labels. Le modèle va pouvoir distinguer les images representant des objets différents sans pour autant savoir ce que ces objets representent.

### Avantages de cette approche

On peut se demander l'intêret d'entraîner un tel modèle. A quoi ce modèle va bien pouvoir servir une fois entraîné ? 

Pour cela, on peut faire une analogie aux modèles de langages. Ces modèles sont d'abord pré-entrainé sur une bonne partie d'internet et ensuite on les *fine-tune* sur une tâche précise (chatbot pour chatGPT par exemple). Pour les images c'est à peu près pareil, les modèles entrainés de manière contrastive sur des milliards d'images peuvent servir de modèles génériques que l'on pourra ensuite *fine-tune* sur des tâches plus spécifiques comme la classification. 

**Note** : Le *fine-tuning* et le *transfer learning* sont abordés en détail dans le cours suivant. En gros, ce sont des techniques pour ré-utiliser un modèle déjà entraîné sur des tâches différentes. 

## Alternative à l'entraînement contrastif pour le non supervisé

Il est important de mentionner que l'entraînement contrastif n'est pas la seule manière de faire de l'entraînement non supervisé sur des images. 

### Autoencodeurs

Nous avons déjà présenté les autoencodeurs dans [le cours 4](../04_Autoencodeurs/README.md) qui peuvent permettre d'apprendre des répresentations d'images pertinentes. L'article [Masked Autoencoders Are Scalable Vision Learners](https://arxiv.org/pdf/2111.06377) démontre que les *masked autoencodeurs* peuvent servir à apprendre des representations d'images très utiles.

![mae](./images/mae.png)


### Generative adversarial network (GAN)

Dans ce cours, nous n'avons pas encore parlé des GAN. Rapidement, il s'agit de réseaux s'entrainant de manière adversaire, un générateur va générer de fausses images et un discriminateur va devoir faire la différence entre une image réelle et une image construite par le générateur. En s'entraînant ensemble, on peut avoir un générateur capable de générer des images très réalistes sans avoir eu besoin d'un dataset labélisé. Il y a quelques années, c'était la méthode la plus utilisées pour la génération d'images (depuis, on utilise plutôt les modèles de diffusion qui sont d'ailleurs non supervisé également).

![gan](./images/gan.png)


### Prédiction de transformation

Une autre approche consiste à prédire une transformation que l'on a appliqué à l'image. On peut par exemple faire une rotation de l'image ([RotNet](https://arxiv.org/pdf/1803.07728)) et entraîner le modèle à la prédire ou bien mélanger notre image à la manière d'un puzzle et entraîner notre modèle à reconstruire l'image ([JigSaw](https://arxiv.org/pdf/1603.09246)). 

![jigsaw](./images/jigsaw.png)

### Distillation auto-supervisée

Plus récemment, des méthodes basées sur la distillation des connaissances utilisent à nouveau des transformations d'images comme dans l'entraînement contrastif mais n'utilisent pas de paires négatives. Pour éviter que le modèle ne *collapse*, diverses techniques sont utilisées. Pour en savoir plus, vous pouvez lire l'article [DINO](https://arxiv.org/pdf/2104.14294).  

**Note** : Le concept de distillation des connaissances sera abordé dans le cours suivant. 

Voici un aperçu des representations apprises par le modèle DINO : 

![dino](./images/dino.png)

Figure extraite de ce [blogpost](https://levelup.gitconnected.com/meta-dino-how-self-supervised-learning-is-changing-computer-vision-1666a5e43dbb).

**Note** : La liste des méthodes non supervisées n'est pas exhaustive mais vous avez maintenant une bonne idée des méthodes existantes. Aussi, les GAN et modèles de diffusion sont non supervisés mais ne sont pas utilisés pour créer des modèles de base pouvant être *fine-tune* sur des tâches plus spécifiques.